In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator') 
# More on this here: https://panel.holoviz.org/reference/widgets/Tabulator.html

import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
# Grab some data from Our World in Data. Go to https://github.com/owid/co2-data/blob/master/owid-co2-data.csv and grab the raw data.
df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

In [3]:
df

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.165,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.163,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50593,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,0.469,0.032,9.596,-0.937,-8.899,...,NaN,0.114,0.001,0.001,0.002,0.0,115.59,28.30,0.910,9.486
50594,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,0.558,0.037,11.795,2.199,22.920,...,NaN,0.114,0.001,0.001,0.002,0.0,118.22,30.83,0.771,6.537
50595,Zimbabwe,2019,ZWE,15354606.0,NaN,0.570,0.037,11.115,-0.681,-5.772,...,NaN,0.113,0.001,0.001,0.002,0.0,117.96,30.53,0.978,8.795
50596,Zimbabwe,2020,ZWE,15669663.0,NaN,0.570,0.036,10.608,-0.507,-4.559,...,NaN,0.112,0.001,0.001,0.002,0.0,NaN,NaN,1.006,9.481


In [4]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [5]:
df[df['country'] == 'World']

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
49810,World,1750,NaN,7.456641e+08,NaN,NaN,NaN,9.351,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49811,World,1751,NaN,NaN,NaN,NaN,NaN,9.351,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49812,World,1752,NaN,NaN,NaN,NaN,NaN,9.354,0.004,0.039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49813,World,1753,NaN,NaN,NaN,NaN,NaN,9.354,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49814,World,1754,NaN,NaN,NaN,NaN,NaN,9.358,0.004,0.039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50077,World,2017,NaN,7.599822e+09,1.104307e+14,1507.923,0.198,36096.738,572.547,1.612,...,100.0,100.0,0.394,1.039,1.511,0.078,48251.879,47031.820,0.004,0.0
50078,World,2018,NaN,7.683790e+09,1.136302e+14,1569.218,0.204,36826.508,729.772,2.022,...,100.0,100.0,0.399,1.058,1.536,0.079,49368.039,47980.469,-0.004,-0.0
50079,World,2019,NaN,7.764951e+09,NaN,1617.507,0.208,37082.559,256.049,0.695,...,100.0,100.0,0.404,1.076,1.561,0.081,49758.230,48116.559,0.000,0.0
50080,World,2020,NaN,7.840953e+09,NaN,1637.537,0.209,35264.086,-1818.472,-4.904,...,100.0,100.0,0.410,1.094,1.585,0.082,NaN,NaN,0.000,0.0


# Data Clean Up

In [6]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!=0, df['gdp']/df['population'],0)

In [7]:
# Make dataframe pipeline interactive
idf = df.interactive()

# CO2 emission over time per continent

In [8]:
# Define the Panel widgets
# Find out more widgets here: panel.holoviz.org/user_guide/Widgets.html
year_slider = pn.widgets.IntSlider(name='Year Slider', start=1750, end=2020, step=5, value=1850)
year_slider

IntSlider(end=2020, name='Year Slider', start=1750, step=5, value=1850)

In [9]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['co2', 'co2_per_capita',],
    button_type = 'success'
)

In [10]:
# Let's create a CO2 Data Pipeline
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica' ]

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [11]:
# Here is what the data pipeline looks like
co2_pipeline.head(n=10)

In [12]:
# Here is a plot of the data pipeline
co2_plot = co2_pipeline.hvplot(x = 'year', by = 'country', y=yaxis_co2, line_width=2, title = 'CO2 Emission by Continent')
co2_plot

# Table of CO2 Emission Over Time By Continent

In [13]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
co2_table

In [14]:
# Let's build a new timeline for CO2 vs. GDP. 
# We will look at countries and not continents this time.

co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [15]:
# Quick preview of the head of the pipeline
co2_vs_gdp_scatterplot_pipeline.head(n=15)

In [16]:
# Now let's build the scatterplot that's based on the pipeline above
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x = 'gdp_per_capita', y = 'co2', by = 'country', size = 80, kind = 'scatter', alpha = 0.7, legend = False, height = 500, width = 500)

# View the scatterplot
co2_vs_gdp_scatterplot

# Bar Chart with CO2 Sources by Continent

In [17]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['coal_co2', 'oil_co2', 'gas_co2'],
    button_type = 'success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica' ]
co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [18]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind = 'bar', x = 'country', y = yaxis_co2_source, title = 'CO2 Source by Continent')
co2_source_bar_plot

# Let's Create the Dashboard

In [19]:
# Layout using the FastListTemplate
# More on the template here: https://panel.holoviz.org/reference/templates/FastListTemplate.html

template = pn.template.FastListTemplate(
    title = 'World CO2 Emission Dashboard',
    sidebar = [
        pn.pane.Markdown("# CO2 Emissions and Climate Change"),
        pn.pane.Markdown("#### Carbon dioxide emissions are believed to be the primary drivers of global climate change."),
        pn.pane.PNG('img/randomimage.png', sizing_mode = 'scale_both'),
        pn.pane.Markdown("## Settings"),
        year_slider],
    main = [
        pn.Row(pn.Column(yaxis_co2, co2_plot.panel(width=700), margin=(0,25)), 
               co2_table.panel(width=500)),
        pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)),
              pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))
    ],
    accent_base_color = "#88d8b0",
    header_background = "#88d8b0",
)

# Time to display the dashboard
template.servable();

# Launch the Dashboard via Terminal

At this point in the process, go to the terminal. Preferably in the same virtual environment as this notebook. If possible, use the terminal console in this instance of Jupyter Lab where the notebook is live. After typing "panel serve thisnotebookname.ipynb" in the terminal, there should come a URL in the terminal that you can then copy and paste in a browser to vide the dashboards.